In [ ]:
!pip install statsmodels==0.11.1
!pip install fbprophet==0.7.1

In [ ]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from statsmodels.tsa.arima.model import ARIMA
from fbprophet import Prophet
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 필사 노트북: 시즌1 이산팀, 2위
# Prophnet + ARIMA

#prophnet 설명
'''
 Prophet은 페이스북에서 공개한 시계열 예측 라이브러리 
 정확도가 높고 빠르며 직관적인 파라미터로 모델 수정이 용이함 
prophnet은 trend, seasonality, holiday의 구성요소로 이루어져있듬
y(t) = g(t) + s(t) + h(t) + error
g(t): piecewise linear or logistic growth curve for modelling non-periodic changes in time series, 비주기적 패턴 
s(t): periodic changes (e.g. weekly/yearly seasonality) , 주기적 패턴
h(t): effects of holidays (user provided) with irregular schedules, 불규칙적 이벤트
ϵ_i: error term accounts for any unusual changes not accommodated by the model, 에러

'''

'''
ARIMA 설명
ARIMA(Auto-regressive Integrated Moving Average) 모형은 시계열 데이터 기반 분석 기법으로 과거지식이나 경험을 바탕으로 한 행동에 따라 경제가 움직이고 있음을 기초로 한다
ARIMA 모형은 과거의 관측 값과 오차를 사용해서 현재의 시계열 값을 설명하는 ARMA(Auto-regressive Moving Average) 모델을 일반화 한 것으로, 분기/반기/연간 단위로 다음 지표를 예측한다거나 주간/월간 단위로 지표를 리뷰하며 트렌드에 이상치가 없는지를 모니터링 하는 데 사용되는 분석 기법이다. ARMA 모델이 안정적 시계열(Stationary Series)에만 적용 가능한 것에 비해, 
분석 대상이 다소 비안정적인 시계열(Non Stationary Series)의 특징을 보여도 적용이 가능하다.
안정적인 시계열이란 시간의 추이와 관계없이 평균 및 분산이 불변하거나 시점 간의 공분산이 기준시점과 무관한 형태의 시계열이다. 시계열이 안정적이지 않을 때는 로그를 이용하거나 차분을 통해 시계열을 안정적으로 변환한 뒤에 분석을 진행한다. 다음 그래프는 안정적인 시계열과 비안정적인 시계열의 몇 가지 예시이다.
'''

In [ ]:
# 데이터 불러오고 확인하기
data_path = '/content/drive/MyDrive/dacon'
train_x_df = pd.read_csv(data_path  + "/train_x_df.csv")
train_y_df = pd.read_csv(data_path  + "/train_y_df.csv")
test_x_df = pd.read_csv(data_path  + "/test_x_df.csv")

def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d

train_x_array = df2d_to_array3d(train_x_df)
train_y_array = df2d_to_array3d(train_y_df)
test_x_array = df2d_to_array3d(test_x_df)

def plot_series(x_series, y_series):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'output_series')
    plt.axhline(1, c = 'red')
    plt.legend()
    
data_col_idx = 1


In [ ]:

import datetime
start_time = '2021-01-31 00:00:00'
start_dt = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

idx = 1121
x_series = train_x_array[idx,:,data_col_idx]
y_series = train_y_array[idx,:,data_col_idx]


In [ ]:
# 모댈 적용 models, models.fit

# 통계모델 ARIMA불러오기
# ARIMA의 모수는 크게 3가지가 있다. 
# AR모형의 Lag을 의미하는 p, MA모형의 Lag을 의미하는 q, 차분(Diffrence)횟수를 의미하는 d 가 그것이다. 
# 보통은 p, d, q의 순서로 쓴다
model = ARIMA(x_series, order=(3,0,1))
fit  = model.fit()
preds1 = fit.predict(1381,1380+120, typ='levels')


x_df = pd.DataFrame()
x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
x_df['y'] = x_series.tolist()

# prophet model
prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=False,
                  weekly_seasonality=False, daily_seasonality=True,
                  changepoint_prior_scale=0.06)
prophet.fit(x_df)

future_data = prophet.make_future_dataframe(periods=120, freq='min')
forecast_data = prophet.predict(future_data)
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)

pred_y = forecast_data.yhat.values[-120:]

plot_series(x_series, y_series)
plt.plot(np.arange(1380, 1380+120), preds1, label = 'ARIMA prediction')
plt.plot(np.arange(1380, 1380+120), pred_y, label = 'Prophet prediction')
plt.legend()
plt.show()


In [ ]:
# buy_quantity(구매량)와 sell time (판매시점 결정)
def array_to_submission(x_array, pred_array):
    # 입력 x_arrry와 출력 pred_arry를 통해서 
    # buy_quantitiy와 sell_time을 결정
    submission = pd.DataFrame(np.zeros([pred_array.shape[0],2], np.int64),
                columns = ['buy_quantity', 'sell_time'])
    submission = submission.reset_index()
    submission.loc[:, 'buy_quantity'] = 0.1
    
    buy_price = []
    for idx, sell_time in enumerate(np.argmax(pred_array, axis = 1)):
        buy_price.append(pred_array[idx, sell_time])
    buy_price = np.array(buy_price)
    # 105% 이상 상승한하고 예측한 sample에 대해서만 100% 매수
    arr1 = (buy_price > 1.05) * 1
    submission.loc[:, 'buy_quantity'] = arr1
    # 모델이 예측값 중 최대 값에 해당하는 시간에 매도
    submission['sell_time'] = np.argmax(pred_array, axis = 1)

    submission.columns = ['sample_id','buy_quantity', 'sell_time']
    return submission

In [ ]:
# 투자 후 금액 계산
def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index

In [ ]:
# test 데이터 학습 및 추론
test_pred_array = np.zeros([test_x_array.shape[0],
                           120])
for idx in tqdm(range(test_x_array.shape[0])):
    try:
        x_series = test_x_array[idx,:,data_col_idx]
        fit = None
        try:
            model = ARIMA(x_series, order=(3,0,1))
            fit  = model.fit()
        except Exception as e:
            model = ARIMA(x_series, order=(4,0,1))
            fit  = model.fit()
            
        x_df = pd.DataFrame()
        x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
        x_df['y'] = x_series.tolist()
        prophet = Prophet(seasonality_mode='multiplicative', 
                          yearly_seasonality=False,
                          weekly_seasonality=False, daily_seasonality=True,
                          changepoint_prior_scale=0.06)
        prophet.fit(x_df)

        preds = fit.predict(1381,1380+120, typ='levels')

        future_data = prophet.make_future_dataframe(periods=120, freq='min')
        forecast_data = prophet.predict(future_data)
        forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)

        pred_y = forecast_data.yhat.values[-120:]
        pred_y_lower = forecast_data.yhat_lower.values[-120:]
        pred_y_upper = forecast_data.yhat_upper.values[-120:]

        max_two_model = np.maximum(pred_y, preds)

        test_pred_array[idx,:] = max_two_model
    except Exception as e:
        print(repr(e))
        print(idx, " 샘플은 수렴하지 않습니다.")
        pass

In [ ]:
submission = array_to_submission(test_x_array, test_pred_array)

submission.buy_quantity.value_counts() 

submission.to_csv(data_path+"/submission.csv", index = False)